In [1]:
import pandas as pd
import numpy as np

# Mostrar floats com duas casas decimas
pd.set_option('display.float_format',  lambda x: '%.2f' % x)
pd.options.display.max_colwidth = 20
pd.options.display.max_columns = 20
pd.options.display.max_rows = 4

In [2]:
# Load stock list
df_magic = pd.read_csv("../data/magic_stocks.csv", parse_dates=["balancing_on"])
df_magic.rank_final = df_magic.rank_final.astype(int)
df_magic

,balancing_on,codneg,nomres,codcvm,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic,market_cap,enterprise_value,earnings_yield,rank_final
0,2011-04-11,TOTS3,TOTVS,19992,2011-01-31 19:05:59,2010-12-31,31.46,179.42,211.67,0.17,994.10,1173.53,0.18,1
1,2011-04-11,ETER3,ETERNIT,5762,2011-03-11 11:18:46,2010-12-31,89.50,-56.72,123.66,0.23,987.18,930.47,0.13,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,2022-04-11,MLAS3,MULTILASER,26034,2022-03-24 23:45:31,2021-12-31,820.54,-362.41,778.48,0.14,4824.77,4462.36,0.17,29
359,2022-04-11,SUZB3,SUZANO S.A.,13986,2022-02-09 18:00:34,2021-12-31,1361.26,58529.58,18180.19,0.16,74120.80,132650.38,0.14,30


In [3]:
# Slice dataframe with columns that will be used
cols = ['balancing_on', 'codneg']
df_magic = df_magic.loc[:, cols]
df_magic

,balancing_on,codneg
0,2011-04-11,TOTS3
1,2011-04-11,ETER3
...,...,...
358,2022-04-11,MLAS3
359,2022-04-11,SUZB3


In [4]:
# List of magic tickers
magic_tickers = list(df_magic['codneg'].unique())
print('Number of tickers =', len(magic_tickers))
print(magic_tickers)

Number of tickers = 115
['TOTS3', 'ETER3', 'VALE5', 'AUTM3', 'RAPT4', 'BRAP4', 'FESA4', 'UGPA4', 'TGMA3', 'JHSF3', 'FHER3', 'HBOR3', 'EZTC3', 'ALPA4', 'CCIM3', 'GRND3', 'MYPK3', 'EVEN3', 'POMO4', 'DIRR3', 'NATU3', 'BEEF3', 'BRKM5', 'GOAU4', 'LREN3', 'MDIA3', 'BISA3', 'FLRY3', 'SLED4', 'BRML3', 'ODPV3', 'SHOW3', 'MRVE3', 'LAME4', 'MGLU3', 'MAGG3', 'RENT3', 'LEVE3', 'SLCE3', 'CSAN3', 'PTBL3', 'EMBR3', 'KLBN4', 'EUCA4', 'PCAR4', 'TIMP3', 'HGTX3', 'RDNI3', 'BEMA3', 'CCRO3', 'GSHP3', 'IGTA3', 'AMAR3', 'CYRE3', 'CVCB3', 'ARZZ3', 'SEER3', 'JBSS3', 'WEGE3', 'ALSC3', 'ANIM3', 'RLOG3', 'FIBR3', 'MRFG3', 'SUZB5', 'BRFS3', 'TCSA3', 'UGPA3', 'POSI3', 'SMLE3', 'FRAS3', 'SGPS3', 'TPIS3', 'SMTO3', 'VULC3', 'CAML3', 'GUAR3', 'VALE3', 'PRIO3', 'TEND3', 'PARD3', 'SMLS3', 'SUZB3', 'TUPY3', 'HAPV3', 'HYPE3', 'CEAB3', 'TRIS3', 'ENAT3', 'UCAS3', 'VIVA3', 'YDUQ3', 'LOGN3', 'MTRE3', 'INTB3', 'TASA4', 'LAVV3', 'PLPL3', 'ALLD3', 'ASAI3', 'KEPL3', 'CURY3', 'PCAR3', 'USIM5', 'LJQQ3', 'RANI3', 'CMIN3', 'SYNE3', 'GG

In [5]:
# Load complete B3 adjusted price data
# s3://aq-dl/HistoricalQuotations/processed/dataset.feather
# Select magic stocks after 2011 (first year with available accounting data)
# Remove columns that will not be used for backtesting
# Daily average stock price (premed) will be used for entering and exiting positions
file_path = "/mnt/aq_disk/data/HistoricalQuotations/processed/dataset.feather"
df_prices = (pd
    .read_feather(file_path)
    .query('codneg == @magic_tickers and datneg >= "2011.01.01"')
    .reset_index(drop=True)
    [['datneg', 'codneg', 'nomres', 'preult', 'premed']]
)
df_prices

,datneg,codneg,nomres,preult,premed
0,2021-04-12,ALLD3,ALLIED,15.92,16.27
1,2021-04-13,ALLD3,ALLIED,15.85,15.96
...,...,...,...,...,...
238213,2022-07-14,YDUQ3,YDUQS PART,14.15,13.95
238214,2022-07-15,YDUQ3,YDUQS PART,14.72,14.44


In [6]:
# Join price dataframe with magic stocks dataframe
# Since there are stocks in multiple periods, this is a one-to-many join
df_prices = df_prices.merge(right=df_magic, how='inner')
df_prices

,datneg,codneg,nomres,preult,premed,balancing_on
0,2021-04-12,ALLD3,ALLIED,15.92,16.27,2021-04-12
1,2021-04-12,ALLD3,ALLIED,15.92,16.27,2022-04-11
...,...,...,...,...,...,...
865064,2022-07-14,YDUQ3,YDUQS PART,14.15,13.95,2020-04-09
865065,2022-07-15,YDUQ3,YDUQS PART,14.72,14.44,2020-04-09


In [7]:
# Create a list of ordered unique balancing dates
values = list(df_prices.balancing_on.sort_values().drop_duplicates())
# Add one extra year to the end of the list
values.append(values[-1] + pd.DateOffset(years=1))
keys = list(range(len(values)))
# Create a dictionary where the keys are the balancing dates
balancing_dict = dict(zip(keys, values))
balancing_dict

{0: Timestamp('2011-04-11 00:00:00'),
 1: Timestamp('2012-04-09 00:00:00'),
 2: Timestamp('2013-04-10 00:00:00'),
 3: Timestamp('2014-04-10 00:00:00'),
 4: Timestamp('2015-04-10 00:00:00'),
 5: Timestamp('2016-04-11 00:00:00'),
 6: Timestamp('2017-04-10 00:00:00'),
 7: Timestamp('2018-04-10 00:00:00'),
 8: Timestamp('2019-04-10 00:00:00'),
 9: Timestamp('2020-04-09 00:00:00'),
 10: Timestamp('2021-04-12 00:00:00'),
 11: Timestamp('2022-04-11 00:00:00'),
 12: Timestamp('2023-04-11 00:00:00')}

In [8]:
# Rank the balancing dates so we have the keys for mapping with the balancing_dict
df_prices['balancing_key'] = df_prices['balancing_on'].rank(method='dense').astype(int)
df_prices

,datneg,codneg,nomres,preult,premed,balancing_on,balancing_key
0,2021-04-12,ALLD3,ALLIED,15.92,16.27,2021-04-12,11
1,2021-04-12,ALLD3,ALLIED,15.92,16.27,2022-04-11,12
...,...,...,...,...,...,...,...
865064,2022-07-14,YDUQ3,YDUQS PART,14.15,13.95,2020-04-09,10
865065,2022-07-15,YDUQ3,YDUQS PART,14.72,14.44,2020-04-09,10


In [9]:
# Map values so that we have the next year balancing dates
df_prices['next_balancing'] = df_prices['balancing_key'].map(balancing_dict)
df_prices.drop(columns='balancing_key', inplace=True)
df_prices

,datneg,codneg,nomres,preult,premed,balancing_on,next_balancing
0,2021-04-12,ALLD3,ALLIED,15.92,16.27,2021-04-12,2022-04-11
1,2021-04-12,ALLD3,ALLIED,15.92,16.27,2022-04-11,2023-04-11
...,...,...,...,...,...,...,...
865064,2022-07-14,YDUQ3,YDUQS PART,14.15,13.95,2020-04-09,2021-04-12
865065,2022-07-15,YDUQ3,YDUQS PART,14.72,14.44,2020-04-09,2021-04-12


In [10]:
# Select prices that are between each of the balancing intervals
df_prices.query('balancing_on <= datneg <= next_balancing', inplace=True)
df_prices.sort_values(['codneg', 'datneg'], inplace=True, ignore_index=True)
df_prices

,datneg,codneg,nomres,preult,premed,balancing_on,next_balancing
0,2021-04-12,ALLD3,ALLIED,15.92,16.27,2021-04-12,2022-04-11
1,2021-04-13,ALLD3,ALLIED,15.85,15.96,2021-04-12,2022-04-11
...,...,...,...,...,...,...,...
83019,2021-04-09,YDUQ3,YDUQS PART,30.57,30.53,2020-04-09,2021-04-12
83020,2021-04-12,YDUQ3,YDUQS PART,30.58,30.59,2020-04-09,2021-04-12


In [11]:
df_prices.to_csv('../data/adjusted_prices.csv', index=False)